CS524: Introduction to Optimization Lecture 35
======================================

### Michael C. Ferris <br><br> Computer Sciences Department <br> University of Wisconsin-Madison

### November 22, 2024

# Stochastic programming: newsvendor problem

In [1]:
import gamspy as gp
import gamspy.math as gpm
from gamspy import Sum, Card

import sys
import numpy as np
import pandas as pd
from statistics import NormalDist

class arguments:
  def __init__(self, s=400, solver='cplex', seed = 0):
    self.s = s
    self.solver = solver
    self.seed = seed
      
args = arguments(s=400)

Calculate the mean value solution

In [2]:
m = gp.Container()

# Purchase costs per unit
c = 0.3
# Return value per unit leftover
r = 0.05
# Revenue per unit sold
q = 2.0

s = gp.Set(m,'s',description='planning scenarios',records = ['mvs'])
d = gp.Parameter(m,'d',domain=s,records=[('mvs',100)],description='random demand')

x = gp.Variable(m,'x','positive',description='Units bought')
y = gp.Variable(m,'y','positive',domain=s,description='Units sold')
z = gp.Variable(m,'z','positive',domain=s,description='Returns')

defDem = gp.Equation(m,'defDem',domain=s,description='demand =  UnitsSold + LostSales')
defDem[s]= d[s] >= y[s]

defRet = gp.Equation(m,'defRet',domain=s,description='Returns(salvage) = UnitsBought - UnitsSold')
defRet[s]= z[s] == x - y[s]

nb = gp.Model(m,'nb',
    equations=m.getEquations(),
    problem=gp.Problem.LP,
    sense=gp.Sense.MAX,
    objective=-c*x + Sum(s, q*y[s] + r*z[s])
)

nb.solve(output=None)

# POST PROCESSING #
results = {}
results['Xmvs'] = x.toValue()

# demand sample for out of sample testing
np.random.seed(args.seed) 
testD = np.random.normal(100,20,3000)
# profit sample
F = np.zeros(np.size(testD))
for i in range(len(testD)):
    F[i] = min((q-c)*results['Xmvs'], (q-r)*testD[i] + (r-c)*results['Xmvs'])

results['Fmvs'] = np.sum(F)/len(testD)
print(f'Mean value soln, x: {results['Xmvs']} f: {results['Fmvs']}')

Mean value soln, x: 100.0 f: 154.30141556119048


In [3]:
# Now do stochastic model

s.setRecords([f's{ind+1}' for ind in range(args.s)])
d.setRecords(np.random.normal(100,20,args.s))
p = gp.Parameter(m,'p',domain=s,
    records= np.array([1/args.s for ind in range(args.s)]))

nb = gp.Model(m,'nb',
    equations=[defDem,defRet],
    problem=gp.Problem.LP,
    sense=gp.Sense.MAX,
    objective=-c*x + Sum(s, p[s]*(q*y[s] + r*z[s]))
)

nb.solve(output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,159.42702716105,801,802,LP,CPLEX,0.001


In [4]:
# POST PROCESSING #
results['Xss'] = x.toValue()

F = np.zeros(np.size(testD))
for i in range(len(testD)):
    F[i] = min((q-c)*results['Xss'], (q-r)*testD[i] + (r-c)*results['Xss'])
results['Fss'] = np.sum(F)/len(testD)
print(f'Stochastic soln({args.s}), x: {results['Xss']} f: {results['Fss']}')

# Closed form stochastic solution

# ss.norm.ppf(0.95, loc=100, scale=20)
CXss = NormalDist(mu=100, sigma=20).inv_cdf((q-c)/(q-r))
print(f'CXss = {CXss}')

# Calculate VSS
vss =  results['Fss'] - results['Fmvs']
print(f'vss = {vss}')

Stochastic soln(400), x: 122.1238294570209 f: 161.08457511581554
CXss = 122.69833235986982
vss = 6.783159554625058
